In [2]:
# Only use "ADNIMERGE.csv" as phenotype and covariate data.

import numpy as np
import pandas as pd

Load sample names of VCF data.

In [7]:
VCFsamplelist = pd.read_csv("ADNI/WGS_Omni2.5M_20140220/vcfsample_list.txt", header = None)
VCFsamplelist_IND = VCFsamplelist.applymap(lambda x: x.split("_")[0]).iloc[:,0]
VCFsamplelist_IND = VCFsamplelist_IND.rename('IND')
VCFsamplelist_ID = VCFsamplelist.applymap(lambda x: x.split("_")[1]).iloc[:,0]
VCFsamplelist_ID = VCFsamplelist_ID.rename('ID')
VCFsamplelist = pd.concat([VCFsamplelist_IND,VCFsamplelist_ID],axis=1)

In [8]:
VCFsamplelist = VCFsamplelist.rename(columns={'IND':'FID', 'ID':'IID'})
#VCFsamplelist_data.to_csv("keep_samples.txt", sep="\t", index=False)
VCFsamplelist = VCFsamplelist.sort_values('IID')
VCFsamplelist

,FID,IID
16,17,0002
581,587,0008
316,319,0015
616,622,0021
688,694,0023
152,154,0031
680,686,0035
614,620,0042
756,763,0047
804,811,0051


In [9]:
# Merge summary data.
ADNI_BASIC_data = pd.read_csv("ADNI/Cov/ADNIMERGE.csv")
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data.iloc[:,4]!="ADNI3"]
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["VISCODE"]=="bl"]

ADNI_BASIC_data.loc[ADNI_BASIC_data["ABETA_bl"]==">1700","ABETA_bl"] = 1700
ADNI_BASIC_data.loc[ADNI_BASIC_data["ABETA_bl"]=="<200","ABETA_bl"] =200
ADNI_BASIC_data = ADNI_BASIC_data.astype({"ABETA_bl": "float"})
ADNI_BASIC_data.loc[ADNI_BASIC_data["TAU_bl"]==">1300","TAU_bl"] = 1300
ADNI_BASIC_data.loc[ADNI_BASIC_data["TAU_bl"]=="<80","TAU_bl"] = 80
ADNI_BASIC_data = ADNI_BASIC_data.astype({"TAU_bl": "float"})
ADNI_BASIC_data.loc[ADNI_BASIC_data["PTAU_bl"]==">120","PTAU_bl"] = 120
ADNI_BASIC_data.loc[ADNI_BASIC_data["PTAU_bl"]=="<8","PTAU_bl"] = 8
ADNI_BASIC_data = ADNI_BASIC_data.astype({"PTAU_bl": "float"})
    
ADNI_BASIC_data = ADNI_BASIC_data[~np.isnan(ADNI_BASIC_data["ABETA_bl"])]
ADNI_BASIC_data = ADNI_BASIC_data[["RID", "VISCODE", "AGE", "PTGENDER", "APOE4", "ABETA_bl","TAU_bl","PTAU_bl", "EXAMDATE"]]

ADNI_BASIC_data = ADNI_BASIC_data.astype({"RID": "int64"})
ADNI_BASIC_data["RID"] = ADNI_BASIC_data["RID"].apply(lambda x:str(x).zfill(4))
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("RID")

ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["RID"].isin(VCFsamplelist["IID"])]

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
def gender2val(df):
    if df["PTGENDER"] == "Male":
        return 1
    elif df["PTGENDER"] == "Female":
        return 0

for rid in ADNI_BASIC_data["RID"].iteritems():
    ridval = rid[1]
    #print(ADNI_BASIC_data.loc[ADNI_BASIC_data["RID"]==rid[1],:])
    ADNI_BASIC_data.loc[ADNI_BASIC_data["RID"]==ridval,"FID"] = VCFsamplelist.loc[VCFsamplelist["IID"]==ridval,"FID"].values
ADNI_BASIC_data["SEX"] = ADNI_BASIC_data.apply(gender2val, axis=1)
ADNI_BASIC_data = ADNI_BASIC_data.rename(columns={'RID': 'IID'})
ADNI_BASIC_data = ADNI_BASIC_data[["FID","IID","ABETA_bl","TAU_bl","PTAU_bl","AGE","SEX","APOE4","EXAMDATE"]]
ADNI_BASIC_data = ADNI_BASIC_data.astype({"FID": "int64"})
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("FID")


ADNI_BASIC_data_cov = ADNI_BASIC_data[["FID","IID","AGE","SEX","APOE4"]]
ADNI_BASIC_data_cov.to_csv("cov_merge_ADNI_CSF.txt", sep="\t", index=False)

ADNI_BASIC_data_pheno = ADNI_BASIC_data[["FID","IID","ABETA_bl","TAU_bl","PTAU_bl"]]
ADNI_BASIC_data_pheno.to_csv("pheno_merge_ADNI_CSF.txt", sep="\t", index=False)

In [11]:
ADNI_BASIC_data

,FID,IID,ABETA_bl,TAU_bl,PTAU_bl,AGE,SEX,APOE4,EXAMDATE
106,1,0056,1048.0,128.20,12.23,69.6,0,0.0,2005-12-22
122,3,0059,1605.0,186.40,16.69,70.9,0,0.0,2005-12-22
12148,5,2237,1700.0,185.30,16.42,65.0,1,0.0,2011-01-26
316,6,0123,1566.0,290.10,26.30,73.3,1,0.0,2006-02-07
563,8,0256,288.4,293.90,27.12,70.1,1,2.0,2006-04-05
3042,10,2264,839.5,234.00,21.16,69.3,1,1.0,2011-03-11
3008,11,2216,530.0,234.00,23.14,68.1,1,1.0,2011-03-10
754,15,0331,588.0,417.10,39.86,64.6,0,2.0,2006-04-06
638,16,0285,844.4,365.10,37.99,65.6,1,1.0,2006-04-03
2126,21,1030,862.4,396.20,39.82,67.4,1,0.0,2006-11-13
